In [13]:
from tinygrad.nn import Conv2d, BatchNorm2d, Tensor, optim
import tinygrad.nn as nn
import torch
from torch.utils.data import DataLoader, Dataset
from tqdm import trange
import matplotlib.pyplot as plt
import pickle
import numpy as np
try:
  from helpers import get_model_size, estimate_loss, normalize_tensor
except ModuleNotFoundError:
  import sys
  sys.path.append("../")
  from helpers import get_model_size, estimate_loss, normalize_tensor

In [10]:
class CiFaData(Dataset):
  def __init__(self, stage="train", transform=None, device="cpu"):
    self.device = device
    self.base_folder = "cifar-10-batches-py"
    self.transform = transform
    if stage == "train":
      batch_collection = [f"data_batch_{i}" for i in range(1, 5)]
    elif stage == "val":
      batch_collection = ["data_batch_5"]
    elif stage == "test":
      batch_collection = ["test_batch"]
    else:
      raise ValueError("Invalid stage, choose from train, val, test.")
    self.x_data = []
    self.y_data = []
    for batch in batch_collection:
      with open(f"../data/cifar-10-batches-py/{batch}", "rb") as f:
        data = pickle.load(f, encoding="latin1") 
        self.x_data.extend(data["data"])
        self.y_data.extend(data["labels"])
    self.y_data = torch.tensor(self.y_data, device=self.device)
    self.x_data = normalize_tensor(torch.tensor(np.vstack(self.x_data).reshape(-1, 3, 32, 32), device=self.device)) # from list to vstack; results in (N, 3, 32, 32)
  def __len__(self):
    return len(self.y_data)
  def __getitem__(self, idx):
    if self.transform:
      return self.transform(self.x_data[idx]), self.y_data[idx]
    return self.x_data[idx], self.y_data[idx]
  

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

train_ds = CiFaData(stage="train", device=device)
val_ds = CiFaData(stage="val", device=device)
test_ds = CiFaData(stage="test", device=device)

# pinning memory, takes cpu data and pins it to the gpu. meaning if I already 
train_loader = DataLoader(train_ds, batch_size=256, shuffle=True) 
val_loader = DataLoader(val_ds, batch_size=256, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=256, shuffle=False)

In [14]:
nn.Module

AttributeError: module 'tinygrad.nn' has no attribute 'Module'

In [15]:
class SubBlock:
  def __init__(self, inchannels, outchannels, stride, kernelsize=3, padding=1):
    self.conv1 = Conv2d(in_channels=inchannels, out_channels=outchannels, kernel_size=kernelsize, padding=padding, stride=stride)
    self.bnorm1 = BatchNorm2d(outchannels)
      nn.ReLU(), # there is no relu module, so this needs to go in the call block ?
      nn.Conv2d(in_channels=outchannels, out_channels=outchannels, kernel_size=kernelsize, padding=padding, stride=1),
      nn.BatchNorm2d(outchannels)
    )
  def __call__(self, x):
    return self.block(x)

AttributeError: module 'tinygrad.nn' has no attribute 'Module'